In [ ]:
# scvelo virtual environment

import scvelo as scv
import numpy as np
import scanpy as sc
import math

In [ ]:
import subprocess

# List all installed packages with their versions
def list_installed_packages():
    result = subprocess.run(['pip', 'list'], stdout=subprocess.PIPE)
    print(result.stdout.decode('utf-8'))

# List all installed packages with their versions in requirements format
def freeze_installed_packages():
    result = subprocess.run(['pip', 'freeze'], stdout=subprocess.PIPE)
    print(result.stdout.decode('utf-8'))

# Execute the functions
list_installed_packages()
freeze_installed_packages()


In [ ]:
n_obs = 1000
n_vars = 20
t_max = 25
alpha = 1.5
beta = .2
gamma = .1
noise_level = 3.5
seed = 520
max_thre = 100

# generate S with Gaussian noise
data = scv.datasets.simulation(n_obs=n_obs,
                               n_vars=n_vars,
                               t_max=t_max,
                               alpha=alpha,
                               beta=beta,
                               gamma=gamma,
                               noise_level=noise_level,
                               random_seed=seed)
S = data.layers['spliced']
U = data.layers['unspliced']
data.layers['true_velocity'] = beta*U-gamma*S
S_lam = math.e**(math.log(max_thre)*S/np.max(S)) - 1
U_lam = math.e**(math.log(max_thre)*U/np.max(U)) - 1

np.random.seed(929)
S_poi = np.random.poisson(lam=[s for s in S_lam])
U_poi = np.random.poisson(lam=[u for u in U_lam])
data.layers['spliced'] = S_poi.copy()
data.layers['unspliced'] = U_poi.copy()
data.X = S_poi.copy() # need to make sure X is the same as spliced matrix

In [ ]:
# https://github.com/theislab/scvelo/issues/1212

scv.pp.log1p(data)
sc.pp.pca(data)
sc.pp.neighbors(data, n_pcs=30, n_neighbors=30)
scv.pp.moments(data, n_pcs=None, n_neighbors=None)
scv.tl.recover_dynamics(data,t_max=25)
scv.tl.velocity_graph(data,vkey="true_velocity")

In [ ]:
scv.pl.velocity_embedding_grid(data,
                               basis='pca',
                               color="true_t",
                               vkey="true_velocity",
                               arrow_length=2,
                               arrow_size=2,
                               min_mass=10)

In [ ]:
scv.pl.velocity_embedding_stream(data,
                               basis='pca',
                               color="true_t",
                               vkey="true_velocity")

Now for scVelo

In [ ]:
data2 = scv.datasets.simulation(n_obs=n_obs,
                               n_vars=n_vars,
                               t_max=t_max,
                               alpha=alpha,
                               beta=beta,
                               gamma=gamma,
                               noise_level=noise_level,
                               random_seed=seed)
S = data2.layers['spliced']
U = data2.layers['unspliced']
S_lam = math.e**(math.log(max_thre)*S/np.max(S)) - 1
U_lam = math.e**(math.log(max_thre)*U/np.max(U)) - 1

np.random.seed(929)
S_poi = np.random.poisson(lam=[s for s in S_lam])
U_poi = np.random.poisson(lam=[u for u in U_lam])
data2.layers['spliced'] = S_poi.copy()
data2.layers['unspliced'] = U_poi.copy()
data2.X = S_poi.copy() # need to make sure X is the same as spliced matrix

In [ ]:
data2.X[0:10,0:10]

In [ ]:
# List of quantiles you want to find
quantiles = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]

# Find the quantiles of the flattened array
quantile_results = np.quantile(data2.X, quantiles)

print(f"The quantiles {quantiles} of the matrix are: {quantile_results}")

In [ ]:
# https://github.com/theislab/scvelo/issues/1212

scv.pp.log1p(data2)
sc.pp.pca(data2)
sc.pp.neighbors(data2, n_pcs=30, n_neighbors=30)
scv.pp.moments(data2, n_pcs=None, n_neighbors=None)

In [ ]:
scv.tl.recover_dynamics(data2)

In [ ]:
scv.tl.velocity(data2, mode='dynamical')

In [ ]:
scv.tl.velocity_graph(data2)

In [ ]:
scv.pl.velocity_embedding_grid(data2,
                               basis='pca',
                               color="true_t",
                               arrow_length=2,
                               arrow_size=2,
                               min_mass=10)

In [ ]:
scv.pl.velocity_embedding_stream(data2,
                               basis='pca',
                               color="true_t")

In [ ]:
scv.tl.velocity_confidence(data2)
keys = 'velocity_confidence'
scv.pl.scatter(data2, c=keys, cmap='coolwarm', perc=[5, 95])